In [2]:

!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F
from torchvision import datasets, transforms

import os

In [16]:
#数据集 处理
def make_dataset(fpath, tpath = 'data'):
    
    
    data = pd.read_csv(fpath) # csv
    os.makedirs(os.path.join(tpath), exist_ok=True)
    for i in range(data.shape[0]):
        if data.shape[1] == 2: #训练集
            fname = os.path.join(tpath, 'trainset')# 训练集目标位置
            os.makedirs(fname, exist_ok=True)
            os.makedirs(os.path.join(fname, data.iloc[i,1]), exist_ok=True)
            shutil.copy(os.path.join(*fpath.split('\\')[:-1], data.iloc[i, 0]), os.path.join(fname, data.iloc[i,1]))
            
        elif data.shape[1] == 1: #测试集
            fname = os.path.join(tpath, 'testset')
            os.makedirs(fname, exist_ok=True)
            print(os.path.join( *fpath.split('\\')[:-1], data.iloc[i, 0]))
            shutil.copy(os.path.join( *fpath.split('\\')[:-1], data.iloc[i, 0]), fname)
        else:
            assert 'make dataset error!'

In [17]:
# make_dataset(os.path.join('..', 'data', 'train.csv'), 'data')

## resnet-

In [33]:

class Residual(nn.Module):
    def __init__(self, input_channels, num_channels,
                use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels, 
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels, 
                               kernel_size=3, padding=1)
        
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels, 
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        
    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [34]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,),
    nn.BatchNorm2d(64),nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding =1))

In [35]:
def resnet_block(input_channels, num_channels, num_residuals, 
                first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels, 
                               use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [36]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

In [37]:
net =nn.Sequential(b1, b2, b3, b4, b5, 
                  nn.AdaptiveAvgPool2d((1,1)),
                  nn.Flatten(), nn.Linear(512, 176))

In [42]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


In [43]:
def train_ch6(net, trian_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('trianing on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', xlim=[1,num_epochs],
                           legend=['train loss', 'trian acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)
    for epoch in range(num_epochs):
        metric = d2l.Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i+1) / num_batches, (train_l, train_acc, None))
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))                        
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}',
         f'test_acc {test_acc:.3f}')
    print(f'{metric[2] *num_epochs / timer.sum():.1f} examples/sec',
         f'on {str(device)}')
            

In [1]:
#图片处理流水线 resize 224, 转为向量 归一化
transform = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])
train_dataset = datasets.ImageFolder(os.path.join('train', 'train'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join('train', 'test'), transform=transform)

train_iter = torch.utils.data.DataLoader(train_dataset,batch_size=32, shuffle=True)
test_iter = torch.utils.data.DataLoader(test_dataset,batch_size=32, shuffle=False)

images = next(iter(test_iter))

NameError: name 'transforms' is not defined

In [31]:
lr, num_epochs, batch_size = 0.05, 10, 32


d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())



ValueError: too many values to unpack (expected 2)